# Phase 2: Memory Seeding & One-Shot Recall
- Load trained model.
- Ingest JSONL/CSV corpora into hippocampal memory.
- Test one-shot recall.

In [ ]:
%pip install -q torch datasets transformers sentencepiece tqdm
import torch, sys
from pathlib import Path
ROOT = Path('/content/repo') if Path('/content/repo').exists() else Path.cwd().parent
sys.path.insert(0, str(ROOT))
from colab_l4_training import get_test_config, main, ingest_jsonl_to_memory, ingest_csv_pairs_to_memory, one_shot_memorize_and_generate

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
# Load model (short run or load checkpoint if you have one)
config = get_test_config()
config.enable_continuous_learning = False
model, losses, cl_orch = main(config_preset='test')
hippocampus = model.hippocampus
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base', legacy=True)
print('Hippocampus memories:', hippocampus.memory_count)

In [ ]:
# Ingest JSONL corpora into episodic memory
jsonl_paths = [
    ROOT/'vocab_src/instruct_55k_clean.jsonl',
    ROOT/'vocab_src/OpenThoughts-114k.jsonl',
    ROOT/'vocab_src/principles_texts.jsonl',
]
for p in jsonl_paths:
    if p.exists():
        stored = ingest_jsonl_to_memory(str(p), tokenizer, model, hippocampus, device, max_items=500)
        print(p.name, 'stored', stored)

# Ingest CSV Q/A pairs (e.g., timeline conversations)
csv_path = ROOT/'vocab_src/timeline_conversations.csv'
if csv_path.exists():
    stored = ingest_csv_pairs_to_memory(str(csv_path), tokenizer, model, hippocampus, device, max_items=500)
    print(csv_path.name, 'stored', stored)

print('Total memories:', hippocampus.memory_count)

In [ ]:
# One-shot recall test
support = "Quantum entanglement links particles across any distance."
prompt = "Explain entanglement to a student"
generated = one_shot_memorize_and_generate(support, prompt, tokenizer, model, hippocampus, device)
print(generated)